In [1]:
import numpy as np 
import pandas as pd 
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

df = pd.read_csv('UniversalBank.csv')

dt_train,dt_test = train_test_split(df,test_size=0.3,shuffle = False)

# calculate error
def error(y, y_pred):
    sum = 0
    for i in range(0, len(y)):
        sum = sum + abs(y[i] - y_pred[i])
    return sum/len(y)  # tra ve trung binh

X_train = dt_train.iloc[:, :8]
y_train = dt_train.iloc[:, 8]
X_test = dt_test.iloc[:, :8]
y_test = dt_test.iloc[:, 8]

min=999999
k = 5
kf = KFold(n_splits=k, random_state=None)
for train_index, validation_index in kf.split(dt_train):
    X_train, X_validation = dt_train.iloc[train_index,:8], dt_train.iloc[validation_index, :8]
    y_train, y_validation = dt_train.iloc[train_index, 8], dt_train.iloc[validation_index, 8]


    svm = SVC().fit(X_train,y_train)
    y_train_pred = svm.predict(X_train)
    y_validation_pred = svm.predict(X_validation)
    y_train = np.array(y_train)
    y_validation = np.array(y_validation)

    sum_error = error(y_train,y_train_pred) + error(y_validation, y_validation_pred)
    
    if(sum_error < min):
        min = sum_error
        regr=svm

y_test_pred = regr.predict(dt_test.iloc[:,:8])
y_test = np.array(dt_test.iloc[:,8])

print("Perceptron:")
print("Accuracy: ",accuracy_score(y_test,y_test_pred))
print("Precision: ",precision_score(y_test,y_test_pred,zero_division=1))# tỉ lệ số điểm true positive trong số những điểm được phân loại là positive
print("recall: ",recall_score(y_test,y_test_pred,zero_division=1))# tỉ lệ số điểm true positive trong số những điểm thực sự là positive
print("f1_score: ",f1_score(y_test,y_test_pred,zero_division=1))

# đự đoán sai
count = 0
for i in range(0,len(y_test_pred)):
    if(y_test[i] != y_test_pred[i]): count += 1

print("Ty le du doan sai : ",count/len(y_test_pred))


Perceptron:
Accuracy:  0.916
Precision:  1.0
recall:  0.0
f1_score:  0.0
Ty le du doan sai :  0.084


In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import Perceptron
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

df = pd.read_csv('UniversalBank.csv')
dt_train,dt_test = train_test_split(df,test_size=0.3,shuffle = False)

# calculate error
def error(y, y_pred):
    sum = 0
    for i in range(0, len(y)):
        sum = sum + abs(y[i] - y_pred[i])
    return sum/len(y)  # tra ve trung binh

X_train = dt_train.iloc[:, :8]
y_train = dt_train.iloc[:, 8]
X_test = dt_test.iloc[:, :8]
y_test = dt_test.iloc[:, 8]

min=999999
k = 5
kf = KFold(n_splits=k, random_state=None)
for train_index, validation_index in kf.split(dt_train):
    X_train, X_validation = dt_train.iloc[train_index,:8], dt_train.iloc[validation_index, :8]
    y_train, y_validation = dt_train.iloc[train_index, 8], dt_train.iloc[validation_index, 8]


    per = Perceptron(alpha= 0.01).fit(X_train,y_train)
    y_train_pred = per.predict(X_train)
    y_validation_pred = per.predict(X_validation)
    y_train = np.array(y_train)
    y_validation = np.array(y_validation)

    sum_error = error(y_train,y_train_pred) + error(y_validation, y_validation_pred)
    
    if(sum_error < min):
        min = sum_error
        regr=per

y_test_pred = regr.predict(dt_test.iloc[:,:8])
y_test = np.array(dt_test.iloc[:,8])

print("Perceptron:")
print("Accuracy: ",accuracy_score(y_test,y_test_pred))
print("Precision: ",precision_score(y_test,y_test_pred,zero_division=1))# tỉ lệ số điểm true positive trong số những điểm được phân loại là positive
print("recall: ",recall_score(y_test,y_test_pred,zero_division=1))# tỉ lệ số điểm true positive trong số những điểm thực sự là positive
print("f1_score: ",f1_score(y_test,y_test_pred,zero_division=1))

# đự đoán sai
count = 0
for i in range(0,len(y_test_pred)):
    if(y_test[i] != y_test_pred[i]): count += 1

print("Ty le du doan sai : ",count/len(y_test_pred))

In [1]:
import numpy as np
from scipy.optimize import Bounds, BFGS
from scipy.optimize import LinearConstraint, minimize
from sklearn import datasets
from math import log, e, sqrt, comb

X, y = datasets.make_blobs(n_samples=100, centers=2, n_features=2, center_box=(0, 10), random_state=1)
y[y == 0] = -1

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape)
print("")

epsilon = 1e-3

def lagrange_dual(alpha, x, y, dim):
    result_v = np.zeros(x.shape[0]).reshape(1, -1)
    ind_sv = np.where(alpha > epsilon)[0]
    for i in ind_sv:
        result_v += alpha[i] * alpha * y[i] * y * ((np.dot(x[i].reshape(-1, 1).T, x.T)) ** dim)
    result_v = 0.5 * result_v - alpha
    return np.sum(result_v.reshape(-1, 1), axis=0)[0]

def optimize_alpha(x, y, C, dim):
    m, n = x.shape
    np.random.seed(1)
    alpha_0 = np.random.rand(m) * C
    linear_constraint = LinearConstraint(y, [0], [0])
    bounds_alpha = Bounds(np.zeros(m), np.full(m, C))
    result = minimize(lagrange_dual, alpha_0, args=(x, y, dim), method='trust-constr', hess=BFGS(), constraints=[linear_constraint], bounds=bounds_alpha)
    alpha = result.x
    return alpha

def calculate_w(alpha, y, x):
    w = alpha.reshape(-1, 1) * y.reshape(-1, 1) * x
    return np.sum(w, axis=0)

def calculate_b(alpha, y, x, w, C, dim):
    C_numeric = C - epsilon
    ind_sv = np.where((alpha > epsilon) & (alpha < C_numeric))[0]
    b = y[np.where((alpha > epsilon) & (alpha < C_numeric))[0]] - (np.dot(x[np.where((alpha > epsilon) & (alpha < C_numeric))[0]], w)) ** dim
    b = b / len(ind_sv)
    return np.sum(b)

def classify_points(x, w, b, dim):
    predicted_labels = np.sum((x * w) ** dim, axis=1) + b
    predicted_labels = np.sign(predicted_labels)
    predicted_labels[predicted_labels == 0] = -1
    return predicted_labels

def misclassification_rate(labels, predictions):
    total = len(labels)
    errors = sum(labels != predictions)
    return errors / total

def Gen_Error(empirical_err, x, VC, delta):
    Gen_err = []
    Gen_err.append(empirical_err + sqrt((8 / x.shape[0]) * log(((4 * (2 * x.shape[0]) ** VC) + 1) / delta)))
    Gen_err.append(empirical_err + sqrt((2 * VC * log((e * x.shape[0]) / VC)) / x.shape[0]) + sqrt(log(1 / delta) / 2 * x.shape[0]))
    return max(Gen_err)

w, b, dim, _ = display_SVM_result(X_train, y_train, 100, 1, False)

y_pred = classify_points(X_test, w, b, dim).astype(int)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))


SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (3507090344.py, line 61)

In [1]:
import time

# Bắt đầu đo thời gian
start_time = time.time()

# Vòng lặp for
for i in range(5000):
    # Thực hiện các hành động của bạn ở đây
    pass

# Kết thúc đo thời gian và in kết quả
end_time = time.time()
print("Thời gian chạy: {} giây".format(end_time - start_time))


Thời gian chạy: 0.0 giây


In [2]:
import pandas as pd

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv("UniversalBank.csv")

# Sử dụng .head(100) để lấy ra 100 hàng đầu tiên
first_100_rows = df.head(100)

# Hoặc bạn có thể sử dụng .iloc[:100]
# first_100_rows = df.iloc[:100]

# In ra 100 hàng đầu tiên
first_100_rows.to_csv("100hang.csv")

In [3]:
import numpy as np
from scipy.optimize import Bounds, BFGS
from scipy.optimize import LinearConstraint, minimize
from sklearn import datasets
from math import log, e, sqrt, comb
import pandas as pd

import time

# Bắt đầu đo thời gian
start_time = time.time()
df = pd.read_csv("100hang.csv")
df['Personal Loan'].replace(0, -1, inplace=True)
# df.head(30)
X = df[['Age','Experience','Income','ZIP Code','Family','CCAvg','Education','Mortgage']].to_numpy()
y = df['Personal Loan']
epsilon = 1e-3


def lagrange_dual(alpha, x, y, dim):
    result_v = np.zeros(x.shape[0]).reshape(1, -1)
    ind_sv = np.where(alpha > epsilon)[0]
    for i in ind_sv:
        # result_v += alpha[i] * alpha * y[i] * y * ((np.dot(x[i].reshape(-1, 1).T, x.T)) ** dim)
        result_v += alpha[i] * alpha * y.to_numpy()[i] * y.to_numpy() * ((np.dot(x[i].reshape(-1, 1).T, x.T)) ** dim)

    result_v = 0.5 * result_v - alpha
    return np.sum(result_v.reshape(-1, 1), axis=0)[0]

def optimize_alpha(x, y, C, dim):
    m, n = x.shape
    np.random.seed(1)
    alpha_0 = np.random.rand(m) * C
    linear_constraint = LinearConstraint(y, [0], [0])
    bounds_alpha = Bounds(np.zeros(m), np.full(m, C))
    result = minimize(lagrange_dual, alpha_0, args=(x, y, dim), method='trust-constr', hess=BFGS(), constraints=[linear_constraint], bounds=bounds_alpha)
    alpha = result.x
    return alpha

def calculate_w(alpha, y, x):
    w = alpha.reshape(-1, 1) * y.reshape(-1, 1) * x
    return np.sum(w, axis=0)

def calculate_b(alpha, y, x, w, C, dim):
    C_numeric = C - epsilon
    ind_sv = np.where((alpha > epsilon) & (alpha < C_numeric))[0]
    b = y[np.where((alpha > epsilon) & (alpha < C_numeric))[0]] - (np.dot(x[np.where((alpha > epsilon) & (alpha < C_numeric))[0]], w)) ** dim
    b = b / len(ind_sv)
    return np.sum(b)

def classify_points(x, w, b, dim):
    predicted_labels = np.sum((x * w) ** dim, axis=1) + b
    predicted_labels = np.sign(predicted_labels)
    predicted_labels[predicted_labels == 0] = -1
    return predicted_labels

def misclassification_rate(labels, predictions):
    total = len(labels)
    errors = sum(labels != predictions)
    return errors / total

def Gen_Error(empirical_err, x, VC, delta):
    Gen_err = []
    Gen_err.append(empirical_err + sqrt((8 / x.shape[0]) * log(((4 * (2 * x.shape[0]) ** VC) + 1) / delta)))
    Gen_err.append(empirical_err + sqrt((2 * VC * log((e * x.shape[0]) / VC)) / x.shape[0]) + sqrt(log(1 / delta) / 2 * x.shape[0]))
    return max(Gen_err)


    
def accuracy_score(labels, predictions):
    total = len(labels)
    correct = sum(labels == predictions)
    return correct / total

from sklearn.model_selection import train_test_split
# dt_train, dt_test = train_test_split(df,  test_size=0.25, random_state=0,shuffle=False)

# X_train = dt_train.iloc[:, :8]
# y_train = dt_train.iloc[:, 8]
# X_test = dt_test.iloc[:, :8]
# y_test = dt_test.iloc[:, 8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, shuffle=False)
# def display_SVM_result(x, y, C, dim, kernel):
#     alpha = optimize_alpha(x, y, C, dim)
#     w = calculate_w(alpha, y, x)
#     b = calculate_b(alpha, y, x, w, C, dim)
    
#     VC = w.shape[0] + dim
#     delta = 0.5
#     predictions = classify_points(x, w, b, dim)
#     empirical_err = misclassification_rate(y, predictions)
#     Gen_Error_Calc = Gen_Error(empirical_err, x, VC, delta)
    
#     # Calculate and print accuracy for both training and test datasets
#     y_pred_train = classify_points(X_train, w, b, dim)
#     accuracy_train = accuracy_score(y_train, y_pred_train)
#     print("Training Accuracy: {:.2f}%".format(accuracy_train * 100))
    
#     y_pred_test = classify_points(X_test, w, b, dim)
#     accuracy_test = accuracy_score(y_test, y_pred_test)
#     print("Test Accuracy: {:.2f}%".format(accuracy_test * 100)) 

def display_SVM_result(x, y, C, dim, kernel):
    alpha = optimize_alpha(x, y, C, dim)
    w = calculate_w(alpha, y, x)
    b = calculate_b(alpha, y, x, w, C, dim)

    VC = w.shape[0] + dim
    delta = 0.5
    predictions = classify_points(x, w, b, dim)
    empirical_err = misclassification_rate(y, predictions)
    Gen_Error_Calc = Gen_Error(empirical_err, x, VC, delta)

    # Calculate and print accuracy for both training and test datasets
    y_pred_train = classify_points(X_train, w, b, dim)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    print("Training Accuracy: {:.2f}%".format(accuracy_train * 100))

    y_pred_test = classify_points(X_test, w, b, dim)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    print("Test Accuracy: {:.2f}%".format(accuracy_test * 100))
 
# Call the function to display the SVM results
display_SVM_result(X_train, y_train, 100, 1, False)
end_time = time.time()
execution_time = end_time - start_time

print(f"Thời gian chạy mã: {execution_time} giây")

AttributeError: 'Series' object has no attribute 'reshape'

In [1]:
import numpy as np
from scipy.optimize import Bounds, BFGS
from scipy.optimize import LinearConstraint, minimize
from sklearn import datasets
from math import log, e, sqrt, comb

X, y = datasets.make_blobs(n_samples=100, centers=2, n_features=2, center_box=(0, 10), random_state=1)
y[y == 0] = -1

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape)
print("")

epsilon = 1e-3

def lagrange_dual(alpha, x, y, dim):
    result_v = np.zeros(x.shape[0]).reshape(1, -1)
    ind_sv = np.where(alpha > epsilon)[0]
    for i in ind_sv:
        result_v += alpha[i] * alpha * y[i] * y * ((np.dot(x[i].reshape(-1, 1).T, x.T)) ** dim)
    result_v = 0.5 * result_v - alpha
    return np.sum(result_v.reshape(-1, 1), axis=0)[0]

def optimize_alpha(x, y, C, dim):
    m, n = x.shape
    np.random.seed(1)
    alpha_0 = np.random.rand(m) * C
    linear_constraint = LinearConstraint(y, [0], [0])
    bounds_alpha = Bounds(np.zeros(m), np.full(m, C))
    result = minimize(lagrange_dual, alpha_0, args=(x, y, dim), method='trust-constr', hess=BFGS(), constraints=[linear_constraint], bounds=bounds_alpha)
    alpha = result.x
    return alpha

def calculate_w(alpha, y, x):
    w = alpha.reshape(-1, 1) * y.reshape(-1, 1) * x
    return np.sum(w, axis=0)

def calculate_b(alpha, y, x, w, C, dim):
    C_numeric = C - epsilon
    ind_sv = np.where((alpha > epsilon) & (alpha < C_numeric))[0]
    b = y[np.where((alpha > epsilon) & (alpha < C_numeric))[0]] - (np.dot(x[np.where((alpha > epsilon) & (alpha < C_numeric))[0]], w)) ** dim
    b = b / len(ind_sv)
    return np.sum(b)

def classify_points(x, w, b, dim):
    predicted_labels = np.sum((x * w) ** dim, axis=1) + b
    predicted_labels = np.sign(predicted_labels)
    predicted_labels[predicted_labels == 0] = -1
    return predicted_labels

def misclassification_rate(labels, predictions):
    total = len(labels)
    errors = sum(labels != predictions)
    return errors / total

def Gen_Error(empirical_err, x, VC, delta):
    Gen_err = []
    Gen_err.append(empirical_err + sqrt((8 / x.shape[0]) * log(((4 * (2 * x.shape[0]) ** VC) + 1) / delta)))
    Gen_err.append(empirical_err + sqrt((2 * VC * log((e * x.shape[0]) / VC)) / x.shape[0]) + sqrt(log(1 / delta) / 2 * x.shape[0]))
    return max(Gen_err)

w, b, dim, _ = display_SVM_result(X_train, y_train, 100, 1, False)

y_pred = classify_points(X_test, w, b, dim).astype(int)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))


(75, 2) (75,)
(25, 2)



NameError: name 'display_SVM_result' is not defined

In [3]:
import numpy as np
from scipy.optimize import Bounds, BFGS
from scipy.optimize import LinearConstraint, minimize
from sklearn import datasets
from math import log, e, sqrt, comb
import pandas as pd

import time

# Bắt đầu đo thời gian
start_time = time.time()
df = pd.read_csv("100hang.csv")
df['Personal Loan'].replace(0, -1, inplace=True)
# df.head(30)
X = df[['Age','Experience','Income','ZIP Code','Family','CCAvg','Education','Mortgage']].to_numpy()
y = df['Personal Loan'].values
epsilon = 1e-3


# def lagrange_dual(alpha, x, y, dim):
#     result_v = np.zeros(x.shape[0]).reshape(1, -1)
#     ind_sv = np.where(alpha > epsilon)[0]
#     for i in ind_sv:
#         # result_v += alpha[i] * alpha * y[i] * y * ((np.dot(x[i].reshape(-1, 1).T, x.T)) ** dim)
#         result_v += alpha[i] * alpha * y.to_numpy()[i] * y.to_numpy() * ((np.dot(x[i].reshape(-1, 1).T, x.T)) ** dim)

#     result_v = 0.5 * result_v - alpha
#     return np.sum(result_v.reshape(-1, 1), axis=0)[0]

def lagrange_dual(alpha, x, y, dim):
    result_v = np.zeros(x.shape[0]).reshape(1, -1)
    ind_sv = np.where(alpha > epsilon)[0]
    for i in ind_sv:
        result_v += alpha[i] * alpha * y[i] * y * ((np.dot(x[i].reshape(-1, 1).T, x.T)) ** dim)

    result_v = 0.5 * result_v - alpha
    return np.sum(result_v.reshape(-1, 1), axis=0)[0]

def optimize_alpha(x, y, C, dim):
    m, n = x.shape
    np.random.seed(1)
    alpha_0 = np.random.rand(m) * C
    linear_constraint = LinearConstraint(y, [0], [0])
    bounds_alpha = Bounds(np.zeros(m), np.full(m, C))
    result = minimize(lagrange_dual, alpha_0, args=(x, y, dim), method='trust-constr', hess=BFGS(), constraints=[linear_constraint], bounds=bounds_alpha)
    alpha = result.x
    return alpha

def calculate_w(alpha, y, x):
    w = alpha.reshape(-1, 1) * y.reshape(-1, 1) * x
    return np.sum(w, axis=0)

def calculate_b(alpha, y, x, w, C, dim):
    C_numeric = C - epsilon
    ind_sv = np.where((alpha > epsilon) & (alpha < C_numeric))[0]
    b = y[np.where((alpha > epsilon) & (alpha < C_numeric))[0]] - (np.dot(x[np.where((alpha > epsilon) & (alpha < C_numeric))[0]], w)) ** dim
    b = b / len(ind_sv)
    return np.sum(b)

def classify_points(x, w, b, dim):
    predicted_labels = np.sum((x * w) ** dim, axis=1) + b
    predicted_labels = np.sign(predicted_labels)
    predicted_labels[predicted_labels == 0] = -1
    return predicted_labels

def misclassification_rate(labels, predictions):
    total = len(labels)
    errors = sum(labels != predictions)
    return errors / total

def Gen_Error(empirical_err, x, VC, delta):
    Gen_err = []
    Gen_err.append(empirical_err + sqrt((8 / x.shape[0]) * log(((4 * (2 * x.shape[0]) ** VC) + 1) / delta)))
    Gen_err.append(empirical_err + sqrt((2 * VC * log((e * x.shape[0]) / VC)) / x.shape[0]) + sqrt(log(1 / delta) / 2 * x.shape[0]))
    return max(Gen_err)


    
def accuracy_score(labels, predictions):
    total = len(labels)
    correct = sum(labels == predictions)
    return correct / total

from sklearn.model_selection import train_test_split
# dt_train, dt_test = train_test_split(df,  test_size=0.25, random_state=0,shuffle=False)

# X_train = dt_train.iloc[:, :8]
# y_train = dt_train.iloc[:, 8]
# X_test = dt_test.iloc[:, :8]
# y_test = dt_test.iloc[:, 8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, shuffle=False)
# def display_SVM_result(x, y, C, dim, kernel):
#     alpha = optimize_alpha(x, y, C, dim)
#     w = calculate_w(alpha, y, x)
#     b = calculate_b(alpha, y, x, w, C, dim)
    
#     VC = w.shape[0] + dim
#     delta = 0.5
#     predictions = classify_points(x, w, b, dim)
#     empirical_err = misclassification_rate(y, predictions)
#     Gen_Error_Calc = Gen_Error(empirical_err, x, VC, delta)
    
#     # Calculate and print accuracy for both training and test datasets
#     y_pred_train = classify_points(X_train, w, b, dim)
#     accuracy_train = accuracy_score(y_train, y_pred_train)
#     print("Training Accuracy: {:.2f}%".format(accuracy_train * 100))
    
#     y_pred_test = classify_points(X_test, w, b, dim)
#     accuracy_test = accuracy_score(y_test, y_pred_test)
#     print("Test Accuracy: {:.2f}%".format(accuracy_test * 100)) 

def display_SVM_result(x, y, C, dim, kernel):
    alpha = optimize_alpha(x, y, C, dim)
    w = calculate_w(alpha, y, x)
    b = calculate_b(alpha, y, x, w, C, dim)

    VC = w.shape[0] + dim
    delta = 0.5
    predictions = classify_points(x, w, b, dim)
    empirical_err = misclassification_rate(y, predictions)
    Gen_Error_Calc = Gen_Error(empirical_err, x, VC, delta)

    # Calculate and print accuracy for both training and test datasets
    y_pred_train = classify_points(X_train, w, b, dim)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    print("Training Accuracy: {:.2f}%".format(accuracy_train * 100))

    y_pred_test = classify_points(X_test, w, b, dim)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    print("Test Accuracy: {:.2f}%".format(accuracy_test * 100))
 
# Call the function to display the SVM results
display_SVM_result(X_train, y_train, 100, 1, False)
end_time = time.time()
execution_time = end_time - start_time

print(f"Thời gian chạy mã: {execution_time} giây")

Training Accuracy: 60.00%
Test Accuracy: 64.00%
Thời gian chạy mã: 189.2664258480072 giây


In [3]:
import numpy as np
from scipy.optimize import Bounds, BFGS
from scipy.optimize import LinearConstraint, minimize
from sklearn import datasets
from math import log, e, sqrt, comb
import pandas as pd

import time

# Bắt đầu đo thời gian
start_time = time.time()
df = pd.read_csv("UniversalBank.csv",nrows=300)
df['Personal Loan'].replace(0, -1, inplace=True)
# df.head(30)
X = df[['Age','Experience','Income','ZIP Code','Family','CCAvg','Education','Mortgage']].to_numpy()
y = df['Personal Loan'].values
epsilon = 1e-3


# lagrange_dual(alpha, x, y, dim): Hàm này tính hàm lagrange đối với các giá trị alpha, dữ liệu x, nhãn y, và chiều dim. Nó sử dụng alpha để tính giá trị của hàm lagrange và trả về kết quả.

# optimize_alpha(x, y, C, dim): Hàm này tối ưu hóa giá trị alpha dựa trên dữ liệu x, nhãn y, hằng số siêu tham số C, và chiều dim. Nó sử dụng phương pháp tối ưu hóa và ràng buộc để tìm giá trị alpha tối ưu và trả về alpha đã được tối ưu.

# calculate_w(alpha, y, x): Hàm này tính vectơ trọng số w dựa trên các giá trị alpha, nhãn y, và dữ liệu x. Vectơ w này sẽ được sử dụng để xác định ranh giới quyết định của mô hình SVM.

# calculate_b(alpha, y, x, w, C, dim): Hàm này tính giá trị b (tham số chặn) của mô hình SVM dựa trên các giá trị alpha, nhãn y, dữ liệu x, vectơ w, hằng số C, và chiều dim. b là một hệ số quan trọng trong việc xác định ranh giới quyết định của mô hình.

# classify_points(x, w, b, dim): Hàm này sử dụng vectơ trọng số w, giá trị b, và chiều dim để dự đoán nhãn cho các điểm dữ liệu trong x. Nó tính toán các điểm dự đoán dựa trên công thức của mô hình SVM và trả về kết quả dưới dạng mảng các nhãn.

# misclassification_rate(labels, predictions): Hàm này tính tỷ lệ lỗi dự đoán bằng cách so sánh nhãn thực tế (labels) với các dự đoán (predictions). Nó trả về tỷ lệ lỗi dự đoán.

# Gen_Error(empirical_err, x, VC, delta): Hàm này tính tỷ lệ lỗi tổng quát (generalization error) dựa trên tỷ lệ lỗi thực nghiệm (empirical_err), số lượng các điểm dữ liệu (x), kích thước VC (VC), và giá trị delta (delta). Nó sử dụng các công thức được định nghĩa để tính toán tỷ lệ lỗi tổng quát dựa trên các thông số này.

# accuracy_score(labels, predictions): Hàm này tính độ chính xác dựa trên nhãn thực tế (labels) và dự đoán (predictions). Nó đếm số lượng dự đoán đúng và tính tỷ lệ độ chính xác của mô hình.

def lagrange_dual(alpha, x, y, dim):
    result_v = np.zeros(x.shape[0]).reshape(1, -1)
    ind_sv = np.where(alpha > epsilon)[0]
    for i in ind_sv:
        result_v += alpha[i] * alpha * y[i] * y * ((np.dot(x[i].reshape(-1, 1).T, x.T)) ** dim)

    result_v = 0.5 * result_v - alpha
    return np.sum(result_v.reshape(-1, 1), axis=0)[0]

def optimize_alpha(x, y, C, dim):
    m, n = x.shape
    np.random.seed(1)
    alpha_0 = np.random.rand(m) * C
    linear_constraint = LinearConstraint(y, [0], [0])
    bounds_alpha = Bounds(np.zeros(m), np.full(m, C))
    result = minimize(lagrange_dual, alpha_0, args=(x, y, dim), method='trust-constr', hess=BFGS(), constraints=[linear_constraint], bounds=bounds_alpha)
    alpha = result.x
    return alpha

def calculate_w(alpha, y, x):
    w = alpha.reshape(-1, 1) * y.reshape(-1, 1) * x
    return np.sum(w, axis=0)

def calculate_b(alpha, y, x, w, C, dim):
    C_numeric = C - epsilon
    ind_sv = np.where((alpha > epsilon) & (alpha < C_numeric))[0]
    b = y[np.where((alpha > epsilon) & (alpha < C_numeric))[0]] - (np.dot(x[np.where((alpha > epsilon) & (alpha < C_numeric))[0]], w)) ** dim
    b = b / len(ind_sv)
    return np.sum(b)

def classify_points(x, w, b, dim):
    predicted_labels = np.sum((x * w) ** dim, axis=1) + b
    predicted_labels = np.sign(predicted_labels)
    predicted_labels[predicted_labels == 0] = -1
    return predicted_labels

def misclassification_rate(labels, predictions):
    total = len(labels)
    errors = sum(labels != predictions)
    return errors / total

def Gen_Error(empirical_err, x, VC, delta):
    Gen_err = []
    Gen_err.append(empirical_err + sqrt((8 / x.shape[0]) * log(((4 * (2 * x.shape[0]) ** VC) + 1) / delta)))
    Gen_err.append(empirical_err + sqrt((2 * VC * log((e * x.shape[0]) / VC)) / x.shape[0]) + sqrt(log(1 / delta) / 2 * x.shape[0]))
    return max(Gen_err)


    
def accuracy_score(labels, predictions):
    total = len(labels)
    correct = sum(labels == predictions)
    return correct / total

from sklearn.model_selection import train_test_split
# dt_train, dt_test = train_test_split(df,  test_size=0.25, random_state=0,shuffle=False)

# X_train = dt_train.iloc[:, :8]
# y_train = dt_train.iloc[:, 8]
# X_test = dt_test.iloc[:, :8]
# y_test = dt_test.iloc[:, 8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, shuffle=False)


def display_SVM_result(x, y, C, dim, kernel):
    alpha = optimize_alpha(x, y, C, dim)
    w = calculate_w(alpha, y, x)
    b = calculate_b(alpha, y, x, w, C, dim)

    VC = w.shape[0] + dim
    delta = 0.5
    predictions = classify_points(x, w, b, dim)
    empirical_err = misclassification_rate(y, predictions)
    Gen_Error_Calc = Gen_Error(empirical_err, x, VC, delta)

    # Calculate and print accuracy for both training and test datasets
    y_pred_train = classify_points(X_train, w, b, dim)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    print("Training Accuracy: {:.2f}%".format(accuracy_train * 100))

    y_pred_test = classify_points(X_test, w, b, dim)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    print("Test Accuracy: {:.2f}%".format(accuracy_test * 100))
 
# Call the function to display the SVM results
display_SVM_result(X_train, y_train, 300, 1, False)
end_time = time.time()
execution_time = end_time - start_time

print(f"Thời gian chạy mã: {execution_time} giây")

Training Accuracy: 67.56%
Test Accuracy: 80.00%
Thời gian chạy mã: 698.6638383865356 giây


In [4]:
import numpy as np
from scipy.optimize import Bounds, BFGS
from scipy.optimize import LinearConstraint, minimize
from sklearn import datasets
from math import log, e, sqrt, comb
import pandas as pd

import time

# Bắt đầu đo thời gian
start_time = time.time()
df = pd.read_csv("UniversalBank.csv",nrows=400)
df['Personal Loan'].replace(0, -1, inplace=True)
# df.head(30)
X = df[['Age','Experience','Income','ZIP Code','Family','CCAvg','Education','Mortgage']].to_numpy()
y = df['Personal Loan'].values
epsilon = 1e-3




def lagrange_dual(alpha, x, y, dim):
    result_v = np.zeros(x.shape[0]).reshape(1, -1)
    ind_sv = np.where(alpha > epsilon)[0]
    for i in ind_sv:
        result_v += alpha[i] * alpha * y[i] * y * ((np.dot(x[i].reshape(-1, 1).T, x.T)) ** dim)

    result_v = 0.5 * result_v - alpha
    return np.sum(result_v.reshape(-1, 1), axis=0)[0]

def optimize_alpha(x, y, C, dim):
    m, n = x.shape
    np.random.seed(1)
    alpha_0 = np.random.rand(m) * C
    linear_constraint = LinearConstraint(y, [0], [0])
    bounds_alpha = Bounds(np.zeros(m), np.full(m, C))
    result = minimize(lagrange_dual, alpha_0, args=(x, y, dim), method='trust-constr', hess=BFGS(), constraints=[linear_constraint], bounds=bounds_alpha)
    alpha = result.x
    return alpha

def calculate_w(alpha, y, x):
    w = alpha.reshape(-1, 1) * y.reshape(-1, 1) * x
    return np.sum(w, axis=0)

def calculate_b(alpha, y, x, w, C, dim):
    C_numeric = C - epsilon
    ind_sv = np.where((alpha > epsilon) & (alpha < C_numeric))[0]
    b = y[np.where((alpha > epsilon) & (alpha < C_numeric))[0]] - (np.dot(x[np.where((alpha > epsilon) & (alpha < C_numeric))[0]], w)) ** dim
    b = b / len(ind_sv)
    return np.sum(b)

def classify_points(x, w, b, dim):
    predicted_labels = np.sum((x * w) ** dim, axis=1) + b
    predicted_labels = np.sign(predicted_labels)
    predicted_labels[predicted_labels == 0] = -1
    return predicted_labels

def misclassification_rate(labels, predictions):
    total = len(labels)
    errors = sum(labels != predictions)
    return errors / total

def Gen_Error(empirical_err, x, VC, delta):
    Gen_err = []
    Gen_err.append(empirical_err + sqrt((8 / x.shape[0]) * log(((4 * (2 * x.shape[0]) ** VC) + 1) / delta)))
    Gen_err.append(empirical_err + sqrt((2 * VC * log((e * x.shape[0]) / VC)) / x.shape[0]) + sqrt(log(1 / delta) / 2 * x.shape[0]))
    return max(Gen_err)


    
def accuracy_score(labels, predictions):
    total = len(labels)
    correct = sum(labels == predictions)
    return correct / total

from sklearn.model_selection import train_test_split
# dt_train, dt_test = train_test_split(df,  test_size=0.25, random_state=0,shuffle=False)

# X_train = dt_train.iloc[:, :8]
# y_train = dt_train.iloc[:, 8]
# X_test = dt_test.iloc[:, :8]
# y_test = dt_test.iloc[:, 8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, shuffle=False)
# def display_SVM_result(x, y, C, dim, kernel):
#     alpha = optimize_alpha(x, y, C, dim)
#     w = calculate_w(alpha, y, x)
#     b = calculate_b(alpha, y, x, w, C, dim)
    
#     VC = w.shape[0] + dim
#     delta = 0.5
#     predictions = classify_points(x, w, b, dim)
#     empirical_err = misclassification_rate(y, predictions)
#     Gen_Error_Calc = Gen_Error(empirical_err, x, VC, delta)
    
#     # Calculate and print accuracy for both training and test datasets
#     y_pred_train = classify_points(X_train, w, b, dim)
#     accuracy_train = accuracy_score(y_train, y_pred_train)
#     print("Training Accuracy: {:.2f}%".format(accuracy_train * 100))
    
#     y_pred_test = classify_points(X_test, w, b, dim)
#     accuracy_test = accuracy_score(y_test, y_pred_test)
#     print("Test Accuracy: {:.2f}%".format(accuracy_test * 100)) 

def display_SVM_result(x, y, C, dim, kernel):
    alpha = optimize_alpha(x, y, C, dim)
    w = calculate_w(alpha, y, x)
    b = calculate_b(alpha, y, x, w, C, dim)

    VC = w.shape[0] + dim
    delta = 0.5
    predictions = classify_points(x, w, b, dim)
    empirical_err = misclassification_rate(y, predictions)
    Gen_Error_Calc = Gen_Error(empirical_err, x, VC, delta)

    # Calculate and print accuracy for both training and test datasets
    y_pred_train = classify_points(X_train, w, b, dim)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    print("Training Accuracy: {:.2f}%".format(accuracy_train * 100))

    y_pred_test = classify_points(X_test, w, b, dim)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    print("Test Accuracy: {:.2f}%".format(accuracy_test * 100))
 
# Call the function to display the SVM results
display_SVM_result(X_train, y_train, 400, 1, False)
end_time = time.time()
execution_time = end_time - start_time

print(f"Thời gian chạy mã: {execution_time} giây")

Training Accuracy: 56.00%
Test Accuracy: 64.00%
Thời gian chạy mã: 1301.3719313144684 giây


In [ ]:
# "Chúng ta sẽ sử dụng phương pháp Lagrange Multipliers để tối thiểu hóa hàm mục tiêu của chúng ta, mà trong trường hợp này là f(w,b) = ||w||^2/2."

# Ý nghĩa: Code sử dụng phương pháp Lagrange Multipliers để tối thiểu hóa hàm mục tiêu f(w,b), mà trong trường hợp này là hàm mất mát của mô hình Support Vector Machine (SVM). Hàm mất mát này có dạng ||w||^2/2, nơi w là vectơ trọng số của mô hình SVM.
# "Phương pháp Lagrange Multipliers thêm hàm mục tiêu vào các ràng buộc của nó (có thể là ràng buộc bằng hoặc ràng buộc không bằng)."

# Ý nghĩa: Phương pháp Lagrange Multipliers thêm hàm mục tiêu (f(w,b)) vào các ràng buộc của nó để tạo ra một hàm Lagrangian. Trong trường hợp này, các ràng buộc là các ràng buộc của mô hình SVM.
# "Chúng ta sau đó nhân mỗi ràng buộc bằng một Lagrange Multiplier, trong trường hợp này là alpha."

# Ý nghĩa: Mỗi ràng buộc của Lagrangian được nhân với một hệ số Lagrange Multiplier (alpha), là một biến để tối ưu hóa. Các alpha này sẽ được tối ưu hóa để đạt được giá trị tối thiểu cho hàm mục tiêu.
# "Chúng ta sau đó lấy đạo hàm của mỗi biến trong hàm Lagrange, và chúng là (w, b, alpha)."

# Ý nghĩa: Đạo hàm của Lagrangian đối với các biến (w, b, alpha) sẽ được tính để tìm điểm cực tiểu. Điều này dẫn đến một bài toán tối ưu hóa bậc hai, nơi chúng ta cố gắng tối ưu hóa các biến w, b, và alpha.
# "Chúng ta sau đó thay thế từng kết quả vào hàm Lagrange của chúng ta, và điều đó sẽ cho chúng ta Lagrangian gấp đôi, là một bài toán lập trình bậc hai (Quadratic programming problem)."

# Ý nghĩa: Bằng cách thay thế các kết quả của đạo hàm vào hàm Lagrange, chúng ta tạo ra một bài toán lập trình bậc hai (Quadratic programming problem) cần được giải quyết. Trong trường hợp này, chúng ta muốn tối ưu hóa Lagrangian để tìm ra giá trị tối thiểu cho hàm mục tiêu của mô hình SVM.
# "Chúng ta có thể sử dụng hàm minimize() trong scipy.optimize để giải quyết bài toán này và lấy được các giá trị alpha, nhưng chúng ta phải khởi tạo chúng trước bằng cách ngẫu nhiên."

# Ý nghĩa: Để giải quyết bài toán tối ưu hóa, chúng ta sử dụng hàm minimize() trong thư viện SciPy để tìm các giá trị tối ưu cho alpha. Trước khi giải quyết, chúng ta phải khởi tạo giá trị ban đầu cho alpha bằng cách chọn ngẫu nhiên.
# "Sau đó chúng ta nhận được các tham số w và b."

# Ý nghĩa: Sau khi giải quyết bài toán tối ưu hóa, chúng ta nhận được các giá trị tối ưu cho alpha, từ đó tính được các tham số w và b của mô hình SVM. Các tham số này xác định đường biên quyết định và ranh giới quyết định của mô hình SVM.

In [ ]:
# lagrange_dual(alpha, x, y, dim): Hàm này tính hàm lagrange đối với các giá trị alpha, dữ liệu x, nhãn y, và chiều dim. Nó sử dụng alpha để tính giá trị của hàm lagrange và trả về kết quả.

# optimize_alpha(x, y, C, dim): Hàm này tối ưu hóa giá trị alpha dựa trên dữ liệu x, nhãn y, hằng số siêu tham số C, và chiều dim. Nó sử dụng phương pháp tối ưu hóa và ràng buộc để tìm giá trị alpha tối ưu và trả về alpha đã được tối ưu.

# calculate_w(alpha, y, x): Hàm này tính vectơ trọng số w dựa trên các giá trị alpha, nhãn y, và dữ liệu x. Vectơ w này sẽ được sử dụng để xác định ranh giới quyết định của mô hình SVM.

# calculate_b(alpha, y, x, w, C, dim): Hàm này tính giá trị b (tham số chặn) của mô hình SVM dựa trên các giá trị alpha, nhãn y, dữ liệu x, vectơ w, hằng số C, và chiều dim. b là một hệ số quan trọng trong việc xác định ranh giới quyết định của mô hình.

# classify_points(x, w, b, dim): Hàm này sử dụng vectơ trọng số w, giá trị b, và chiều dim để dự đoán nhãn cho các điểm dữ liệu trong x. Nó tính toán các điểm dự đoán dựa trên công thức của mô hình SVM và trả về kết quả dưới dạng mảng các nhãn.

# misclassification_rate(labels, predictions): Hàm này tính tỷ lệ lỗi dự đoán bằng cách so sánh nhãn thực tế (labels) với các dự đoán (predictions). Nó trả về tỷ lệ lỗi dự đoán.

# Gen_Error(empirical_err, x, VC, delta): Hàm này tính tỷ lệ lỗi tổng quát (generalization error) dựa trên tỷ lệ lỗi thực nghiệm (empirical_err), số lượng các điểm dữ liệu (x), kích thước VC (VC), và giá trị delta (delta). Nó sử dụng các công thức được định nghĩa để tính toán tỷ lệ lỗi tổng quát dựa trên các thông số này.

# accuracy_score(labels, predictions): Hàm này tính độ chính xác dựa trên nhãn thực tế (labels) và dự đoán (predictions). Nó đếm số lượng dự đoán đúng và tính tỷ lệ độ chính xác của mô hình.